## Data prep

In [54]:
# Load libraries

import pandas as pd
import numpy as np
import glob
pd.options.plotting.backend = "plotly"
import plotly.io; plotly.io.renderers.default = "notebook_connected"

In [2]:
# Merge files and delete duplicates

files = glob.glob('raw data/*.csv')
all_dfs = []
for f in files:
    _df = pd.read_csv(f, parse_dates = ['DATE'],
                      dtype = {'PER/ACRE':str, 'ASS VAL':str},
                      na_values=['', ' ', 'N/A'])
    all_dfs.append(_df)
    
df = pd.concat(all_dfs, ignore_index=True)
df['ASS VAL'] = df['ASS VAL'].str.replace(',','').astype(float)
df['PER/ACRE'] = df['PER/ACRE'].str.replace(',','').astype(float)

df = df.drop_duplicates()

In [3]:
# Standardize LSDs 

df['LEGAL_STD'] = df.LEGAL.str.replace('LSD 0', 'LSD')

In [4]:
# Calculate price difference between sales

df = df.sort_values(['LEGAL_STD', 'DATE'])
df['PRICE_CHANGE'] = df.groupby('LEGAL_STD').PRICE.diff()
df['ACRE_PRICE_CHANGE'] = df.groupby('LEGAL_STD')['PER/ACRE'].diff()

## Analysis

In [12]:
# Define display columns

cols = ['LEGAL', 'RM', 'DATE', 'PRICE', 'PER/ACRE', 'VENDOR', 'PURCHASER']

#### Basic stats

Number of records

In [17]:
len(df)

38451

Time span of data

In [21]:
df.DATE.describe()

<ipython-input-21-4ac38c483eeb>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.DATE.describe()


count                   38451
unique                   5873
top       2014-04-30 00:00:00
freq                       98
first     1993-10-02 00:00:00
last      2021-02-05 00:00:00
Name: DATE, dtype: object

15 biggest single sales

In [13]:
df.sort_values('PRICE', ascending = False).head(15)[cols]

,LEGAL,RM,DATE,PRICE,PER/ACRE,VENDOR,PURCHASER
20004,SW-07-28-09-W2,275 - Insinger,2019-12-10,4500000,3132.83,John Zuchkan,John Zuchkan et al
21543,NE-27-28-08-W2,275 - Insinger,2005-09-09,4051350,25320.94,Yellowhead Pork Producers Ltd.,BIG SKY FARMS INC.
27818,NW-01-32-08-W2,305 - Invermay,2002-10-04,4000000,25000.00,Big Sky Farms Inc.,PRAIRIE HOG PRODUCTION INC.
2031,NE-03-08-23-W2,070 - Key West,2002-10-04,4000000,25157.23,Big Sky Farms Inc.,PRAIRIE HOG PRODUCTION INC.
17185,Parcel# 164702842,250 - Last Mountain Valley,2019-12-09,3762000,836000.00,Victoria & Kent Rhodes,Kent Rhodes et al
27667,SE-12-33-07-W2,305 - Invermay,2004-05-13,3700000,23125.00,Big Sky Farms Inc.,PRAIRIE HOG PRODUCTION INC.
1921,SE-18-08-22-W2,070 - Key West,2003-06-18,3550000,22327.04,Big Sky Farms Inc.,BIG SKY FINISHERS (NO. 1) INC.
1922,NW-11-08-22-W2,070 - Key West,2003-06-18,3550000,22327.04,Big Sky Farms Inc.,BIG SKY FINISHERS (NO. 1) INC.
17184,NW-24-26-22-W2,250 - Last Mountain Valley,2019-12-09,3056400,2147.06,Terry Hamilton,Murray Hamilton et al
32049,NW-08-52-16-W2,488 - Torch River,2020-11-30,2526100,17360.32,TOPSOIL FARM LAND MANAGEMENT (,J & J WINNY HOLDINGS LTD.


15 most expensive per-acre sales


In [14]:
df.sort_values('PER/ACRE', ascending = False).head(15)[cols]

,LEGAL,RM,DATE,PRICE,PER/ACRE,VENDOR,PURCHASER
14548,Parcel# 152172299,247 - Kellross,2018-01-02,50000,5000000.00,VILLAGE OF LESTOCK,RURAL MUNICIPALITY OF KELLROSS
14555,Parcel# 152172299,247 - Kellross,2017-12-01,42450,4245000.00,CORNERSTONE CREDIT UNION,VILLAGE OF LESTOCK
33023,Parcel# 135354898,488 - Torch River,2017-10-03,370000,3363636.36,Quintin & Leanne Wismer,Quint Stacey & Kari Stevenson
32909,Parcel# 203329201,488 - Torch River,2018-02-13,479400,2996250.00,Randy & Shirley Zary,Randy & Shirley Zary
33019,Parcel# 145200945,488 - Torch River,2017-10-19,380000,2923076.92,Donald & Renee Mohr,Katie & Jesse Hupaelo
32937,Parcel# 135355079,488 - Torch River,2018-01-15,300000,2727272.73,Cindy Tyson,TREVOR & CINDY TYSON
32741,Parcel# 135354843,488 - Torch River,2018-07-25,290000,2636363.64,Brad & Jill McClements,Brad McClements
32191,Parcel# 153789182,488 - Torch River,2020-05-11,250000,2272727.27,Aaron & Kara Krahn,TAMMY STANKOWSKI
32271,Parcel# 203329199,488 - Torch River,2020-02-18,215000,1791666.67,Randy & Shirley Zary,Christopher & Nancy Komperdo
32114,Parcel# 135355068,488 - Torch River,2020-10-20,170000,1545454.55,Debra Gane,Debra & Scott Remeshylo


15 buyers who spent the most and the amounts they paid.


In [83]:
df.groupby('PURCHASER').sum().PRICE.sort_values(ascending = False).head(15)

PURCHASER
ANDJELIC LAND INC.                27268145
J & J WINNY HOLDINGS LTD.         23954578
Topsoil Farm Land Mgmt (II)       22688525
Maxcrop Landing Ltd.              16124110
Murray Hamilton et al             15772000
INPUT CAPITAL CORP.               12505374
PRAIRIE HOG PRODUCTION INC.       11700000
Robert Andjelic                   10241000
HRM the Queen (Canada)             9577731
CAROLYNN WALLIN                    9514690
John Zuchkan et al                 9000000
102053876 SASKATCHEWAN LTD.        8805756
SS UNIMIX AGRICULTURE CORPORAT     7696200
211413 Saskatchewan Inc.           7412232
Marv Wahl et al                    7200000
Name: PRICE, dtype: int64

15 buyers who bought the most acres and how many acres they got. It looks like the government is identified in differnet ways (HMQ, HQM(Canada), etc. These would need to be standardized to get more accurate totals.

In [84]:
df.groupby('PURCHASER').sum().ACRES.sort_values(ascending = False).head(15)

PURCHASER
HRM the Queen (Canada)         46662.89
Maxcrop Landing Ltd.           29607.50
ANDJELIC LAND INC.             26875.89
211413 Saskatchewan Inc.       20628.33
Topsoil Farm Land Mgmt (II)    19880.04
STAN DELORME, ET AL            17310.85
HRM the Queen (Sask)           16212.33
Robert Andjelic                14974.94
Murray Hamilton et al          14288.60
J & J WINNY HOLDINGS LTD.      13604.51
HMQ                            11664.65
HMQ (Canada)                   10429.68
HMQ(Canada)                    10411.45
DOUGLAS & BRENDA HANNAH        10032.35
COWESSESS TLE HOLDINGS INC.     9821.00
Name: ACRES, dtype: float64

Biggest change in per-acre price from one sale to the next. It seems like a lot of these lots stayed in the family or moved between farms.

In [27]:
df.sort_values('PRICE_CHANGE', ascending = False).head(15)[cols + ['PRICE_CHANGE']]

,LEGAL,RM,DATE,PRICE,PER/ACRE,VENDOR,PURCHASER,PRICE_CHANGE
20004,SW-07-28-09-W2,275 - Insinger,2019-12-10,4500000,3132.83,John Zuchkan,John Zuchkan et al,4000000.0
21543,NE-27-28-08-W2,275 - Insinger,2005-09-09,4051350,25320.94,Yellowhead Pork Producers Ltd.,BIG SKY FARMS INC.,3979350.0
27818,NW-01-32-08-W2,305 - Invermay,2002-10-04,4000000,25000.00,Big Sky Farms Inc.,PRAIRIE HOG PRODUCTION INC.,3958000.0
2031,NE-03-08-23-W2,070 - Key West,2002-10-04,4000000,25157.23,Big Sky Farms Inc.,PRAIRIE HOG PRODUCTION INC.,3940000.0
27667,SE-12-33-07-W2,305 - Invermay,2004-05-13,3700000,23125.00,Big Sky Farms Inc.,PRAIRIE HOG PRODUCTION INC.,3628000.0
1922,NW-11-08-22-W2,070 - Key West,2003-06-18,3550000,22327.04,Big Sky Farms Inc.,BIG SKY FINISHERS (NO. 1) INC.,3496000.0
1921,SE-18-08-22-W2,070 - Key West,2003-06-18,3550000,22327.04,Big Sky Farms Inc.,BIG SKY FINISHERS (NO. 1) INC.,3488000.0
17185,Parcel# 164702842,250 - Last Mountain Valley,2019-12-09,3762000,836000.00,Victoria & Kent Rhodes,Kent Rhodes et al,3344000.0
17184,NW-24-26-22-W2,250 - Last Mountain Valley,2019-12-09,3056400,2147.06,Terry Hamilton,Murray Hamilton et al,2716800.0
33703,NW-08-52-16-W2,488 - Torch River,2015-06-03,1789155,12295.75,Torch River Agr Corp,Topsoil Farm Land Mgmt (II),1714787.0


Total sales per year


In [55]:
df.groupby(df.DATE.dt.year).PRICE.sum().plot.bar()

Average change in per-acre price per year

In [56]:
df.groupby(df.DATE.dt.year).ACRE_PRICE_CHANGE.mean().plot.bar()

2014 is a huge outlier, let's take a close look. Here are the 15 sales that year with the biggest price change

In [28]:
big_changes_2014 = df[df.DATE.dt.year == 2014].sort_values('ACRE_PRICE_CHANGE', ascending = False).head(15)
big_changes_2014[cols]

,LEGAL,RM,DATE,PRICE,PER/ACRE,VENDOR,PURCHASER
12159,SW-13-24-07-W2,215 - Stanley,2014-09-25,72000,720000.00,Donald Tebb et al,Donald & Noeleen Tebb
29172,LSD 9-09-34-12-W2,336 - Sasman,2014-12-08,12520,178857.14,William & Mary Kulyk,William Kulyk
12161,NW-13-24-07-W2,215 - Stanley,2014-09-25,60100,15774.28,Donald Tebb et al,Noeleen & Donald Tebb
34094,LSD 11-13-52-19-W2,488 - Torch River,2014-04-01,600000,8854.78,Carman Long,Carman Long et al
33947,SW-16-51-14-W2,488 - Torch River,2014-11-13,535500,9916.67,Curtis & Brenda Deutsch,JODY & SHONA FUNK
6314,NW-20-21-05-W2,184 - Grayson,2014-03-13,932500,5835.42,Randy Thiedig,SS UNIMIX AGRICULTURE CORPORAT
9488,SW-03-22-03-W2,213 - Saltcoats,2014-03-05,896400,5603.55,STEPHEN & LINDA ROPER,SS UNIMIX AGRICULTURE CORPORAT
17723,SW-15-25-22-W2,250 - Last Mountain Valley,2014-08-20,520000,3242.91,Barrie & Barbara Gwillim,Barbara Gwillim
6302,NE-19-21-05-W2,184 - Grayson,2014-03-13,437500,2762.52,Randy Thiedig,SS UNIMIX AGRICULTURE CORPORAT
23303,NE-04-30-11-W2,276 - Foam Lake,2014-03-05,416500,2608.18,Ronald & Rita Zadorozniak,Julian Evanochko et al


Let's isolate those properties and look at past sales. Look at property LSD 9-09-34-12-W2. It was sold by William Kulyk for \\$1 in 2012, then bought by William & Mary Kulyk for \\$1, then bought back by William Kulyk for \\$12520 in 2014. Is this anomalous?

In [98]:
df[df.LEGAL_STD.isin(big_changes_2014.LEGAL_STD)][['LEGAL_STD', 'DATE', 'PRICE', 'ACRE_PRICE_CHANGE', 'VENDOR', 'PURCHASER']]

,LEGAL_STD,DATE,PRICE,ACRE_PRICE_CHANGE,VENDOR,PURCHASER
36881,LSD 11-13-52-19-W2,2000-09-15,16500,NaN,Frank/Edna/David Pagan,Carman Long
34094,LSD 11-13-52-19-W2,2014-04-01,600000,8442.28,Carman Long,Carman Long et al
29559,LSD 9-09-34-12-W2,2012-04-19,1,NaN,William Kulyk,LAWRENCE HASKEY
29367,LSD 9-09-34-12-W2,2013-05-22,1,0.00,LAWRENCE HASKEY,William & Mary Kulyk
29172,LSD 9-09-34-12-W2,2014-12-08,12520,178842.85,William & Mary Kulyk,William Kulyk
24926,NE-04-30-11-W2,2003-12-08,40000,NaN,Olga Zadorozniak,Ronald & Rita Zadorozniak
23303,NE-04-30-11-W2,2014-03-05,416500,2358.18,Ronald & Rita Zadorozniak,Julian Evanochko et al
8703,NE-19-21-05-W2,1994-12-20,26000,NaN,Ottilia Thiedig,Randy Thiedig
6302,NE-19-21-05-W2,2014-03-13,437500,2600.02,Randy Thiedig,SS UNIMIX AGRICULTURE CORPORAT
4849,NW-12-11-22-W2,2002-02-12,10000,NaN,Dwite Demerse,Dwite & Linda Demerse


Let's look at stats by RM: number of sales, total amount of sales, and average price per sale. Saltcoats seems to be an outlier with a higher average sales price.

In [33]:
import seaborn as sns

cm = sns.light_palette("blue", as_cmap=True)

stats_by_rm = df.groupby('RM').PRICE.agg(['count','sum', 'mean']).sort_values('mean', ascending = False)
stats_by_rm.style.background_gradient(cmap=cm)

,count,sum,mean
RM,,,
213 - Saltcoats,2738,216124279,78935.090942
488 - Torch River,6511,443296319,68084.214253
215 - Stanley,2629,177350070,67459.136554
336 - Sasman,3571,238377241,66753.637917
250 - Last Mountain Valley,2848,180694797,63446.206812
305 - Invermay,2058,124118221,60310.117104
275 - Insinger,2486,148294291,59651.766291
276 - Foam Lake,3917,225295128,57517.265254
184 - Grayson,2988,168852836,56510.319946


Now let's explore the relationship between acreage and price over time. The chart below show each sale in the data, coloure by RM. Use the slider to cycle through the years. Hover over dots for details.

For the most part, there's a straight relationship between acre and price. Most parcels are around 140-170 acres. But in some years, in some RMs, we see big outliers.

Examples:

* In 2008, a lot of parcels were sold for \\$1.
* In 2011 in Invernay, there were 160-acre parcels sold far above the average for that size.
* Torch River seems to have more under-valued parcels.

In [57]:
import plotly.express as px

px.scatter(df, x = 'ACRES', y = 'PRICE', color = 'RM', 
           animation_frame = sorted(df.DATE.dt.year), range_y = (1, df.PRICE.max()+10000),
           range_x = (1, df.ACRES.max()+100), log_x = True, log_y = True, height = 600,
           template = 'plotly_white')

In [58]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>